In [3]:
from collections import defaultdict as ddict
from typing import DefaultDict, Set
import itertools

import freeman as fm

import pandas as pd
import numpy as np
import networkx as nx

In [53]:
tegtrade = pd.read_csv(
    "csv/results.csv",
     usecols=("team_1", "team_2", "event_id", "date"),
)

In [54]:
tegtrade.loc[:, "date"] = pd.to_datetime(tegtrade["date"], format="%Y-%m-%d")

tegtrade.loc[:, "team_1"] = tegtrade["team_1"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)
tegtrade.loc[:, "team_2"] = tegtrade["team_2"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)

In [71]:
tegtrade = tegtrade[tegtrade["date"] >= "2020-01-01"]


In [72]:
teams = set()
for _, val in tegtrade.iterrows():
    teams.add(tuple(sorted([val["team_1"], val["team_2"]])))

In [81]:
uteams = set(tegtrade["team_1"].unique())
uteams.update(tegtrade["team_2"].unique())

In [74]:
with open("teams.gml", "w+") as f:
    gml = "graph [\n    directed 0\n"
    for team in uteams:
        gml += "    node [\n"
        gml += f"        id \"{team}\"\n"
        gml += "    ]\n"
    c = 0
    for team in teams:
        gml += "    edge [\n"
        gml += f"        source \"{team[0]}\"\n"
        gml += f"        target \"{team[1]}\"\n"
        gml += "    ]\n"
    gml += "]\n"
    f.write(gml)

In [ ]:
bruno.loc[:, "date"] = pd.to_datetime(bruno["date"], format="%Y-%m-%d")
bruno.loc[:, "player_name"] = bruno["player_name"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)
bruno = bruno[bruno["date"] >= "2018-01-01"]

In [ ]:
bruno.cabeca = bruno.head
bruno.cabeca()

In [ ]:
upid_rating = bruno.groupby(["player_id", "player_name"])["rating"]
median = upid_rating.median()
median.name = "median"
mean = upid_rating.mean()
mean.name = "mean"

helionp = pd.concat([median, mean], axis=1)
helionp.ceara = helionp.head
helionp.ceara()

In [ ]:
teams: DefaultDict[str, Set[str]] = ddict(set)
_ = bruno.groupby(["team", "match_id"])["player_id"].apply(
    lambda x: teams[x.name[0]].update(x.values)
)

In [77]:
g = fm.load('teams.gml')

In [78]:
g.set_all_nodes(size=10, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.25))

In [79]:
g.move("kamada_kawai")

In [80]:
g.draw()